# 2. About memory storage

In [172]:
import torch

In [173]:
points = torch.tensor([[4.0, 1.0],
                       [5.0, 3.0],
                       [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

👆

PyTorch 在其最新版本中已经弃用了 `TypedStorage` 类，而只保留了 `UntypedStorage` 类。`TypedStorage` 类是一个旧的类，它允许你创建一个特定类型的存储，如 `FloatStorage` 或 `LongStorage`。然而，PyTorch 的开发者决定弃用这个类，因为它增加了代码的复杂性，而且在大多数情况下并不需要。

所以会有:

```bash
/tmp/ipykernel_24202/484287676.py:4: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()
```

的提示

值得注意的是, 如果将其改换成 `points.untyped_storage()` 时输出和原输出远不一样:

```bash
 0
 0
 128
 64
 0
 0
 128
 63
 0
 0
 160
 64
 0
 0
 64
 64
 0
 0
 0
 64
 0
 0
 128
 63
[torch.storage.UntypedStorage(device=cpu) of size 24]
```


In [174]:
second_point = points[1] # 这是将points的第二行赋值给second_point, 即[5.0, 3.0]
second_point

tensor([5., 3.])

In [175]:
second_point.storage_offset()

2

👆

`second_point.storage_offset()` 返回的 `2` 是指存储中的第三个元素, 即:

```bash
 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]
```

中的 `5.0`, 即第 `2` 个数开始. 

同理的, 如果是:

```python
>>> first_point = points[0]
>>> first_point.storage_offset()
```

则会返回 `0`. 对应 `4.0`, 即第 `0` 个数开始.

In [176]:
second_point.size(), second_point.shape

(torch.Size([2]), torch.Size([2]))

👆

这二者是可以互换的, 区别在于 `.size()` 是一个方法, 而 `.shape` 是一个属性.

In [177]:
second_point[0] = 10

In [178]:
second_point, points

(tensor([10.,  3.]),
 tensor([[ 4.,  1.],
         [10.,  3.],
         [ 2.,  1.]]))

👆

对此, 我们可以看到: 对于 `second_point` 的修改, 会影响到 `points` 中的值. 也就是说 `second_point` 和 `points` 共享了同一个存储空间.

---

由此, 对于数据的复制不能简单用 `=` 来赋值, 而是要用 `.clone()` 方法来进行复制.

👇

In [179]:
another_point = points[1].clone()

another_point, points

(tensor([10.,  3.]),
 tensor([[ 4.,  1.],
         [10.,  3.],
         [ 2.,  1.]]))

In [180]:
another_point.storage_offset()

0

👆

其返回为 `0`, 而不是 `2`, 侧面说明了 `another_point` 不再是和 `points` 共享一块内存空间, 而是新开辟了一块.

In [181]:
second_point = points[1]
second_point.stride(), points.stride()

((1,), (2, 1))

👆

`stride()`, 顾名思义, 就是步长. 也就是说, 从一个元素到下一个元素, 需要跳过多少个元素.

对于 `points` 来说, 由于其是一个 `2 * 1` 的矩阵, 而 `stride()` 的方法是同一维度下来说的, 也就是, 当列上的元素需要跳到下一个维度的时候, 需要跳过 `2` 个元素

```
→ [ 4.,  1.],
→ [10.,  3.],
  [ 2.,  1.]
```

而从列的维度上来说时, 则只需要跳过 `1` 个元素

```
  ⬇    ⬇
[ 4.,  1.],
[10.,  3.],
[ 2.,  1.]
```

同理的, 对于 `secon_point` 来说, 对于其只有一个维度, 那么其 `stride()` 的值就是 `1`.

```
  ⬇    ⬇
[10.,  3.]
```

In [182]:
# 转置
points_t = points.t()
points_t

tensor([[ 4., 10.,  2.],
        [ 1.,  3.,  1.]])

但是, 值得注意的是, 这个 `.t()` 方法并内有进行内存的拷贝, 其内存的 `id`仍是一样的, 也就是说, 他们共享了同一块内存空间.

👇

In [183]:
id(points.storage()) == id(points_t.storage())

False

In [184]:
points_t.stride()

(1, 2)

对于更高维的矩阵的 `stride()`: 

👇

In [185]:
some_t = torch.ones(3, 4, 5)
some_t.stride()

(20, 5, 1)

👆

也就是, 我们可以这样计算: stride就是, 本元素的下面的元素相乘之和.

$$
tensor = (x_1, x_2 \cdots x_{n-1}, x_n) \\
\begin{aligned}
\text{strid} _{1} &= x_2 \times x_3 \cdots x_n \\
\text{strid} _{2} &= x_3 \times x_4 \cdots x_n \\
& \vdots \\
\text{strid} _{n-1} &= x_n \\
\text{strid} _{n} &= 1
\end{aligned}
$$

In [186]:
some_t_t = some_t.transpose(0, 2)
# 值得注意的是:
# transpose() 需要的矩阵是 <= 2 维的, 所以对于更高维的矩阵的转置, 需要指定某两个维度
# 且 t() 和 transpose() 是不一样的, t() 是严格的二维转换, 而 transpose() 可以处理任意维度, 通过指定两个维度
some_t_t.shape

torch.Size([5, 4, 3])

In [187]:
some_t_t.stride()

(1, 5, 20)

👆

不理解, 等之后再看.

In [ ]:
points.is_contiguous()

In [ ]:
points_t.is_contiguous()

```python
points.is_contiguous()
points_t.is_contiguous()
```

其中第一个是 `ture`, 第二个是 `false`, 也就是说, `point` 连续而 `points_t` 不连续.

理解就是对于内存中存的数据:

```bash
 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
```

来说, `point` 可以按顺序一路读下去, 而 `points_t` 则需要反复横跳.